In [278]:
import pandas as pd
import missingno as mgn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
# Export model
import os
import joblib
from sklearn.ensemble import RandomForestClassifier
import re
from sklearn.preprocessing import OneHotEncoder

In [ ]:

#* PARMAS
NEW_VERSION=False

# Encodage des variables categorielles

In [300]:
# # Create the encoder
# encoder = OneHotEncoder()  # drop='first' similar to drop_first=True

# # Fit and transform data
# categorical_cols = ['transmission', 'fuel_type', 'brand', 'model', 'origin', 'condition', 'first_owner']
# encoded_data = encoder.fit_transform(df[categorical_cols])

# # Get feature names
# feature_names = encoder.get_feature_names_out(categorical_cols)

# # Create a DataFrame with the encoded data
# encoded_df = pd.DataFrame(encoded_data, columns=feature_names, index=df.index)

# # Combine with the non-encoded columns
# final_df = pd.concat([df.drop(categorical_cols, axis=1), encoded_df], axis=1)

df.to_csv("../data/cleaned_data_set.csv",index=False)


# Normalization

In [301]:

scaler = StandardScaler()
df[['price', 'mileage', 'tax_horsepower', 'car_age', 'number_of_doors']] = scaler.fit_transform(
    df[['price', 'mileage', 'tax_horsepower', 'car_age', 'number_of_doors']]
)


# Separation des donnees (train | test) + Entrainement du modele

In [302]:
X = df.drop("price", axis=1)
y = df["price"]

print(X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


Index(['model_year', 'transmission', 'fuel_type', 'mileage', 'brand', 'model',
       'number_of_doors', 'origin', 'first_owner', 'tax_horsepower',
       'condition', 'abs', 'airbags', 'multimedia', 'backup_camera',
       'air_conditioning', 'esp', 'aluminum_wheels', 'speed_limiter',
       'onboard_computer', 'parking_sensors', 'cruise_control',
       'leather_seats', 'navigation_gps', 'sunroof', 'remote_central_locking',
       'power_windows', 'price_log', 'car_age'],
      dtype='object')


ValueError: could not convert string to float: 'Manuelle'

In [ ]:
# Directory to save models
if NEW_VERSION:
    
    MODEL_DIR = "."
    os.makedirs(MODEL_DIR, exist_ok=True)

    def get_next_model_version():
        existing_models = [f for f in os.listdir(MODEL_DIR) if re.match(r"v\d+\.pkl", f)]
        versions = [int(re.findall(r'\d+', fname)[0]) for fname in existing_models]
        next_version = max(versions) + 1 if versions else 1
        return f"v{next_version}.pkl"


    # Save with version
    model_filename = get_next_model_version()
    joblib.dump(model, os.path.join(MODEL_DIR, model_filename))
    print(f"Model saved as {model_filename}")


Model saved as v1.pkl


# Evaluation du modele 

In [ ]:
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

MAE: 0.024829234850177018
R2 Score: 0.9358914727430943
